TUTORIAL PARA CONVERTER ESCALAS

In [6]:
import pandas as pd

In [7]:
!pip install dms2dec -q

In [8]:
from dms2dec.dms_convert import dms2dec

In [9]:
dados = pd.read_excel('https://raw.githubusercontent.com/cletofreire/converting_geographic_coordinates/main/data_convert.xlsx',header=0)
dados2 = pd.read_excel('https://raw.githubusercontent.com/cletofreire/converting_geographic_coordinates/main/data_convert.xlsx',header=0)


In [10]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Latitude   200 non-null    object
 1   Longitude  200 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [11]:
#criando um id
dados.reset_index(inplace = True)
dados = dados.rename(columns ={'index':'id'})

In [12]:
# id para os dados2
dados2.reset_index(inplace = True)
dados2 = dados.rename(columns ={'index':'id'})

In [13]:
#Observando se há vazio
print(dados.isnull().sum())


id           0
Latitude     0
Longitude    0
dtype: int64


In [14]:
#Observando as linhas que possuem dados vazios
linhas_vazias = dados[dados.isnull().any(axis=1)]
print(linhas_vazias)

Empty DataFrame
Columns: [id, Latitude, Longitude]
Index: []


In [15]:
dados2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         200 non-null    int64 
 1   Latitude   200 non-null    object
 2   Longitude  200 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.8+ KB


In [16]:
#Selecionando apenas os dados que estão em grau_minuto_segundo
#Latitude
dados['Latitude_GMS'] = dados['Latitude'].astype(str)
dados= dados[dados['Latitude_GMS'].str.contains("[°'\"']", regex=True)]

In [17]:
#Longitude
dados['Longitude_GMS'] = dados['Longitude'].astype(str)
dados= dados[dados['Longitude_GMS'].str.contains("[°'\"']", regex=True)]


In [18]:
# Criei uma função onde substitui os valores desejados para ficar no formato limpo
def remover_os_simbolos(remover):
    return remover.replace('°', ' ').replace ('\'', ' ').replace('\"', ' ')

In [19]:
dados['Latitude_GMS'] = dados['Latitude_GMS'].apply(remover_os_simbolos)
dados['Longitude_GMS'] = dados['Longitude_GMS'].apply(remover_os_simbolos)


In [20]:
# convertendo para Graus_decimaism
dados['Latitude_GD'] = dados['Latitude_GMS'].apply(dms2dec)
dados['Longitude_GD'] = dados['Longitude_GMS'].apply(dms2dec)


In [21]:
dados.head(3)

,id,Latitude,Longitude,Latitude_GMS,Longitude_GMS,Latitude_GD,Longitude_GD
0,0,"01°23'45""","72°34'56""",01 23 45,72 34 56,12345.0,723456.0
1,1,"02°34'56""","63°45'34""",02 34 56,63 45 34,23456.0,634534.0
2,2,"03°45'23""","54°23'12""",03 45 23,54 23 12,34523.0,542312.0


In [22]:
def adicionar_sinal(valor):
    if valor < 0:
        return valor
    else:
        return '-' + str(valor)

In [23]:
# aplicar a função lambda para adicionar um sinal de menos na frente de todos os valores em 'Latitude_GD' e 'Longitude_GD' que não possuem um sinal de menos
dados['Latitude_GD'] = dados['Latitude_GD'].apply(lambda x: adicionar_sinal(x))
dados['Longitude_GD'] = dados['Longitude_GD'].apply(lambda x: adicionar_sinal(x))

In [24]:
dados.head(3)

,id,Latitude,Longitude,Latitude_GMS,Longitude_GMS,Latitude_GD,Longitude_GD
0,0,"01°23'45""","72°34'56""",01 23 45,72 34 56,-12345.0,-723456.0
1,1,"02°34'56""","63°45'34""",02 34 56,63 45 34,-23456.0,-634534.0
2,2,"03°45'23""","54°23'12""",03 45 23,54 23 12,-34523.0,-542312.0


In [25]:
# Unindo os dois dataframes usando a coluna 'id' como chave e uma junção externa
dados_unidos = dados2.merge(dados, on='id', how='outer')



In [26]:
# Preenchendo os valores faltantes de Latitude_GD com os valores correspondentes de Latitude
dados_unidos['Latitude_GD'] = dados_unidos['Latitude_GD'].fillna(dados_unidos['Latitude_x'])



In [27]:
# Preenchendo os valores faltantes de Longitude_GD com os valores correspondentes de Longitude
dados_unidos['Longitude_GD'] = dados_unidos['Longitude_GD'].fillna(dados_unidos['Longitude_x'])



In [28]:
# Descartando as colunas 'Latitude' e 'Longitude' se desejado
dados_unidos = dados_unidos.drop(['Latitude_x', 'Longitude_x'], axis=1)



In [29]:
# Mostrando o dataframe unido
dados_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             200 non-null    int64 
 1   Latitude_y     146 non-null    object
 2   Longitude_y    146 non-null    object
 3   Latitude_GMS   146 non-null    object
 4   Longitude_GMS  146 non-null    object
 5   Latitude_GD    200 non-null    object
 6   Longitude_GD   200 non-null    object
dtypes: int64(1), object(6)
memory usage: 12.5+ KB


In [30]:
dados_unidos.head()

,id,Latitude_y,Longitude_y,Latitude_GMS,Longitude_GMS,Latitude_GD,Longitude_GD
0,0,"01°23'45""","72°34'56""",01 23 45,72 34 56,-12345.0,-723456.0
1,1,"02°34'56""","63°45'34""",02 34 56,63 45 34,-23456.0,-634534.0
2,2,"03°45'23""","54°23'12""",03 45 23,54 23 12,-34523.0,-542312.0
3,3,"04°56'34""","45°34'56""",04 56 34,45 34 56,-45634.0,-453456.0
4,4,"05°12'34""","36°45'23""",05 12 34,36 45 23,-51234.0,-364523.0


In [31]:
# Save
dados_unidos.to_excel('dados_convertidos.xlsx', index=False)
